In [2]:
# 1.0 Installs pyspark, spark-nlp and findspark

!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

# !bash colab.sh
# -p is for pyspark
# -s is for spark-nlp
# !bash colab.sh -p 3.1.1 -s 3.0.1
# by default they are set to the latest

--2021-12-24 17:53:31--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 17:53:31--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 17:53:32--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [3]:
#1.1  To check contents of colab.sh, just download 
#     colab.sh and examine its contents:

!wget http://setup.johnsnowlabs.com/colab.sh 

! cat /content/colab.sh

--2021-12-24 17:54:11--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 17:54:11--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 17:54:11--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [4]:
import pandas as pd
import numpy as np

# 2.1 spark related
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType,StructField,StructType
#Replace part of string with another string
from pyspark.sql.functions import regexp_replace

# 2.2 Spark-nlp related
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [5]:
spark = sparknlp.start()

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
pathToFoler = "/content/drive/MyDrive/BDDA/"
fileName = "emotions.csv"

In [8]:
df = pd.read_csv(pathToFoler+fileName)

In [9]:
df.head()

,Sno,text,emotion
0,1,i didnt feel humiliated,sadness
1,2,i can go from feeling so hopeless to so damned...,sadness
2,3,im grabbing a minute to post i feel greedy wrong,anger
3,4,i am ever feeling nostalgic about the fireplac...,love
4,5,i am feeling grouchy,anger


In [10]:
df['emotion'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [11]:
new = df[['text','emotion']].copy()
cor_data =spark.createDataFrame(new)

In [12]:
cor_data.show(truncate=False)
cor_data.count()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|text                                                                                                                                                                                                                       |emotion |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|i didnt feel humiliated                                                                                                                                                                                                    |sadness |
|i can go from feeling so hopeless to so damned hopeful just from being arou

16000

In [13]:
schema = StructType([ \
                     StructField("Sno",StringType(),True), \
                     StructField("text",StringType(),True), \
                     StructField("emotion",StringType(),True), \
                  
  ])

In [14]:
train = spark.read  \
                 .option("quote", "\"") \
                 .option('escape', "\"") \
                 .option("multiLine", "true")  \
                 .option("schema" , schema)  \
                 .option("header", "true") \
                 .csv(pathToFoler + fileName)

In [15]:
train.show()

+---+--------------------+--------+
|Sno|                text| emotion|
+---+--------------------+--------+
|  1|i didnt feel humi...| sadness|
|  2|i can go from fee...| sadness|
|  3|im grabbing a min...|   anger|
|  4|i am ever feeling...|    love|
|  5|i am feeling grouchy|   anger|
|  6|ive been feeling ...| sadness|
|  7|ive been taking o...|surprise|
|  8|i feel as confuse...|    fear|
|  9|i have been with ...|     joy|
| 10| i feel romantic too|    love|
| 11|i feel like i hav...| sadness|
| 12|i do feel that ru...|     joy|
| 13|i think it s the ...|   anger|
| 14|i feel low energy...| sadness|
| 15|i have immense sy...|     joy|
| 16|i do not feel rea...|     joy|
| 17|i didnt really fe...| sadness|
| 18|i feel pretty pat...| sadness|
| 19|i started feeling...| sadness|
| 20|i now feel compro...|    fear|
+---+--------------------+--------+
only showing top 20 rows



In [16]:
from pyspark.sql.functions import col

# 5.3.1
train.groupBy("emotion") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
| emotion|count|
+--------+-----+
|     joy| 5362|
| sadness| 4666|
|   anger| 2159|
|    fear| 1937|
|    love| 1304|
|surprise|  572|
+--------+-----+



In [17]:
(trainingData, testData) = train.randomSplit([0.8, 0.2], seed = 100)

# 6.0.1
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))


Training Dataset Count: 12825
Test Dataset Count: 3175


In [18]:
# 6.1
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# 6.2    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# 6.3      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# 6.4
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

# 6.5
lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [23]:
# 7.0
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
     .setCaseSensitive(False)

# 7.1
embeddingsSentence = SentenceEmbeddings() \
     .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
   .setPoolingStrategy("AVERAGE")

# 7.2
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("emotion")\
      .setMaxEpochs(100)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

# 7.3
clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
           glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [24]:
# 8.0 Train 
%%time

clf_pipelineModel = clf_pipeline.fit(trainingData)

CPU times: user 1.57 s, sys: 184 ms, total: 1.76 s
Wall time: 4min 54s


In [25]:
preds = clf_pipelineModel.transform(testData)

In [26]:
preds.select('emotion','text',"class.result").show(10, truncate=80)

+-------+--------------------------------------------------------------------------------+---------+
|emotion|                                                                            text|   result|
+-------+--------------------------------------------------------------------------------+---------+
|    joy|i started out feeling really optimistic and driven for this paper coz it was ...|    [joy]|
|sadness|i designed the retirement detox course for all those people who are maybe fee...|    [joy]|
|sadness|i need to do the best i possibly can do and even when i get out at i feel too...|[sadness]|
|    joy|i am still spinning from all the activities but also feeling invigorated and ...|    [joy]|
|    joy|i drove us to the car parts place and terry feels like im safe to drive again...|    [joy]|
|    joy|i have to squint with a magnifying glass to read it i chose the little oxford...|[sadness]|
|sadness|                              i feel embarrassment and shame of being victimized|[

In [27]:
preds_df = preds.select('emotion','text',"class.result").toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])




In [28]:
from sklearn.metrics import classification_report

# 9.5
print (classification_report( preds_df['result'], preds_df['emotion']))

              precision    recall  f1-score   support

       anger       0.51      0.50      0.50       406
        fear       0.00      0.00      0.00         0
         joy       0.84      0.61      0.71      1491
        love       0.00      0.00      0.00         0
     sadness       0.75      0.56      0.64      1278
    surprise       0.00      0.00      0.00         0

    accuracy                           0.57      3175
   macro avg       0.35      0.28      0.31      3175
weighted avg       0.76      0.57      0.65      3175



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
